In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
from torch.cuda import amp

import sys
import helper

from skimage import io
from PIL import Image

In [2]:
import os
import gc
import cv2
import copy
import time
import random
from PIL import Image

from tqdm import tqdm

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
# Hyperparametes
in_channel = 3
num_classes = 100
learning_rate = 1e-3
batch_size = 64
num_epochs = 2
load_model = True
num_workers = 0

In [5]:
kaggle_path = "../input/petfinder-pawpularity-score/"
root = kaggle_path
# Load data
train_df = pd.read_csv(kaggle_path + 'train.csv')
test_df = pd.read_csv(kaggle_path + 'test.csv')
print(train_df.shape)
print(test_df.shape)
n_test_samples = test_df.shape[0]
train_data = kaggle_path + 'train'
test_data = kaggle_path + 'test'

(9912, 14)
(8, 13)


In [6]:
class PicSet(Dataset):
    def __init__(self, root, train=False, transform = None):
        self.train=train
        self.imgs = os.listdir(root)
        self.root = root
        self.transforms = transform

        self.csv_data = pd.read_csv(root + '.csv')


    def __getitem__(self, index):
        img_path = self.root + '/'+self.csv_data['Id'][index] + '.jpg'
        pil_img = Image.open(img_path)
        if self.transforms:
            data = self.transforms(pil_img)
        else:
            pil_img = np.asarray(pil_img)
            data = torch.from_numpy(pil_img)
        if self.train:
            label = torch.tensor(self.csv_data['Pawpularity'][index])
            return data, label
        else:
            return data


    def __len__(self):
        return len(self.imgs)

In [7]:
def load_pic(train_path, test_path):
    transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ConvertImageDtype(torch.float),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])
    train_dataset = PicSet(train_path,train=True, transform = transform)
    pic_num = len(train_dataset)
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset,
                                                               [int(pic_num * 0.9), pic_num - int(pic_num * 0.9)])
    # print('train:', len(train_dataset), 'val:', len(val_dataset))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=num_workers)

    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             num_workers=num_workers)

    test_dataset = PicSet(test_path)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                              num_workers=num_workers)
    return train_loader, val_loader, test_loader

In [8]:
train_loader, val_loader, test_loader = load_pic(train_data, test_data)

In [9]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("Saving Checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print("Loading Checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    

In [ ]:
# Load pretrain model and modify it if available
if load_model:
    model = torchvision.models.densenet161(pretrained=False)
else:
    model = torchvision.models.densenet161(pretrained=True)
    
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(nn.BatchNorm1d(2208), nn.ReLU(inplace=True), nn.Dropout(p=0.6), nn.Linear(2208, 64),
                                 nn.BatchNorm1d(64), nn.ReLU(), nn.Linear(64, 1))
for param in model.classifier.parameters():
    param.require_grad = True
    
model.to(device)

if load_model:
    load_checkpoint(torch.load("../input/pawpularity-trained-model-dense161/my_checkpoint.pth.tar"))
    #load_checkpoint(torch.load("../input/petfinder-pawpularity-score/my_checkpoint.pth.tar"))

In [ ]:
# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.classifier.parameters(), weight_decay=4e-4, lr=1e-3)
reduce_lr = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.6, verbose=True)


# Train Network
for epoch in range(num_epochs):
    batch_train_loss = []
    batch_val_loss = []
    model.train()
    with tqdm(total=len(train_loader)) as t:
        for batch_idx, (data, target) in enumerate(train_loader):
            #Get data to cuda if possible
            data = data.to(device=device)
            target = target.to(device=device)
            target = target.to(torch.float32)

            # forward pass
            scores = model(data)
            loss = criterion(scores, target)
            batch_train_loss.append(loss.item())
            # backward pass
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()
            t.set_postfix(loss=loss.item())
            t.update(1)

    checkpoint = {'state_dict':model.state_dict()}
    save_checkpoint(checkpoint)
    
    model.eval()
    for batch_idx, (data, target) in enumerate(val_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        scores = model(data)
        loss = criterion(scores, target)
        batch_val_loss.append(loss.item())
    
    print(f"Average train loss:{np.mean(batch_train_loss)}")
    print(f"Average val loss:{np.mean(batch_val_loss)}")
    reduce_lr.step()


In [12]:
test_scores = []
for batch_idx, (data) in enumerate(test_loader):
    data = data.to(device)
    model.eval()
    with torch.no_grad():
        scores = model(data)
        scores = np.array(scores.cpu().detach().numpy())
        test_scores.extend(list(np.ceil(np.max(scores, axis=1))))

In [ ]:
submission = pd.DataFrame({"Id":test_df.Id.values, "Pawpularity":test_scores})
submission.to_csv("submission.csv", index=False)

In [13]:
print(submission.head())

                                 Id  Pawpularity
0  4128bae22183829d2b5fea10effdb0c3         18.0
1  43a2262d7738e3d420d453815151079e         16.0
2  4e429cead1848a298432a0acad014c9d         18.0
3  80bc3ccafcc51b66303c2c263aa38486         17.0
4  8f49844c382931444e68dffbe20228f4         17.0
